<a href="https://colab.research.google.com/github/Anuragpandey2005/Knowledge-Assistant-Local-Retrieval-AI-Agent/blob/main/Foxo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# step 1 :Install required libraries


In [ ]:
!pip install sentence-transformers faiss-cpu pdfplumber transformers python-dotenv requests reportlab


# Step 2: Import dependencies

In [ ]:
import os
import re
import pdfplumber
import numpy as np
import faiss
from dotenv import load_dotenv
from sentence_transformers import SentenceTransformer, util
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer


# Step 3: Configuration

In [ ]:
# Step 3: Configuration
load_dotenv()
DOCS_DIR = "foxo_docs"
CHUNK_SIZE = 300
CHUNK_OVERLAP = 50
SIMILARITY_THRESHOLD = 0.6

# Step 4: Enhanced document creation

In [ ]:
def create_sample_documents():
    os.makedirs(DOCS_DIR, exist_ok=True)

    # PDF with multiple pages
    from reportlab.lib.pagesizes import letter
    from reportlab.platypus import SimpleDocTemplate, Paragraph
    from reportlab.lib.styles import getSampleStyleSheet

    def create_pdf(filename, content):
        doc = SimpleDocTemplate(filename, pagesize=letter)
        styles = getSampleStyleSheet()
        story = [Paragraph(page, styles["Normal"]) for page in content]
        doc.build(story)

    create_pdf(
        f"{DOCS_DIR}/security.pdf",
        [
            "Security Policy\n• AES-256 encryption\n• Mandatory 2FA for all access",
            "Data Protection\n• Daily backups\n• GDPR compliance standards"
        ]
    )

    with open(f"{DOCS_DIR}/benefits.txt", "w") as f:
        f.write("Employee Benefits:\n- 25 vacation days/year\n- Comprehensive health insurance")

# Step 5: Improved document loading with page tracking

In [ ]:
# Step 5: Improved document loading with page tracking
def load_documents():
    documents = []
    for filename in os.listdir(DOCS_DIR):
        path = os.path.join(DOCS_DIR, filename)
        if filename.endswith(".txt"):
            with open(path, "r") as f:
                documents.append((f.read(), filename, 1))
        elif filename.endswith(".pdf"):
            with pdfplumber.open(path) as pdf:
                for page_num, page in enumerate(pdf.pages, 1):
                    text = page.extract_text() or ""
                    documents.append((text, filename, page_num))
    return documents

# Step 6: Better text splitting with overlap

In [ ]:
def split_documents(documents):
    chunks = []
    for content, filename, page in documents:
        words = content.split()
        for i in range(0, len(words), CHUNK_SIZE - CHUNK_OVERLAP):
            chunk = " ".join(words[i:i+CHUNK_SIZE])
            chunks.append((chunk, f"{filename} (page {page})"))
    return chunks

# Step 7: Embedding system

In [ ]:


def create_vector_store(chunks):
    embedder = SentenceTransformer("all-MiniLM-L6-v2")
    embeddings = embedder.encode([text for text, _ in chunks])

    index = faiss.IndexFlatL2(embeddings.shape[1])
    index.add(np.array(embeddings, dtype="float32"))
    return index, embedder

# Step 8: Retrieval with threshold

In [ ]:
def retrieve(query, index, embedder, chunks, k=3):
    query_embed = embedder.encode([query])
    distances, indices = index.search(query_embed.astype("float32"), k)

    results = []
    for i, score in zip(indices[0], distances[0]):
        if score < SIMILARITY_THRESHOLD:
            results.append((chunks[i][0], chunks[i][1]))

    return results if results else [chunks[indices[0][0]]]


# Step 9: LLM Setup for T5

In [ ]:
def setup_llm():
    from transformers import T5ForConditionalGeneration, T5Tokenizer

    tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-base")
    model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-base")

    return pipeline(
        "text2text-generation",
        model=model,
        tokenizer=tokenizer,
        max_length=200,
        temperature=0.3,
        repetition_penalty=1.2
    )


# Step 10: RAG Pipeline


In [ ]:
def answer_question(query, index, embedder, chunks, qa_pipeline):

    if "weather" in query.lower():
        return get_weather(query.split()[-1])
    if any(word in query for word in ["calculate", "math", "solve"]):
        return calculate(query)

    # Document retrieval
    docs = retrieve(query, index, embedder, chunks)

    # Answer generation
    context = "\n".join([f"[Source {i+1}: {fn}]\n{text}"
                       for i, (text, fn) in enumerate(docs)])

    prompt = f"""Answer this question using only the context. Cite sources:

Context:
{context}

Question: {query}
Answer:"""

    try:
        response = qa_pipeline(prompt, max_length=200)[0]['generated_text']
        return response.strip()
    except Exception as e:
        return f"Error generating answer: {str(e)}"


# Step 11: Final RAG pipeline

In [ ]:

def answer_question(query, index, embedder, chunks, qa_pipeline):

    if "weather" in query.lower():
        return get_weather(query.split()[-1])
    if any(word in query for word in ["calculate", "math", "solve"]):
        return calculate(query)

    # Document retrieval
    docs = retrieve(query, index, embedder, chunks)

    # Answer generation
    context = "\n".join([f"[Source {i+1}: {fn}]\n{text}"
                       for i, (text, fn) in enumerate(docs)])

    prompt = f"""Use this context to answer. Cite sources like [Source 1].
If unsure, say "I don't know".

Context:
{context}

Question: {query}
Answer:"""

    try:
        response = qa_pipeline(prompt)[0]['generated_text']
        return response.split("Answer:")[-1].strip()
    except Exception as e:
        return f"Error generating answer: {str(e)}"

# Step 12: Main execution

In [ ]:

if __name__ == "__main__":

    create_sample_documents()
    raw_docs = load_documents()
    chunks = split_documents(raw_docs)
    index, embedder = create_vector_store(chunks)
    qa_pipeline = setup_llm()

    # CLI Interface
    print("🦊 FOXO Knowledge Assistant\n")
    while True:
        try:
            query = input("Your question (type 'exit' to quit): ")
            if query.lower() in ["exit", "quit"]:
                break

            answer = answer_question(query, index, embedder, chunks, qa_pipeline)
            print(f"\nAssistant: {answer}\n{'─'*50}\n")

        except KeyboardInterrupt:
            break